<a href="https://colab.research.google.com/github/machiguchi-masaki/pokeca/blob/main/decklist_ver0_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 初期設定値及び、googledriveのマウント
from google.colab import drive
drive.mount('/content/drive')

#デッキコード
deckcode = 'kVbFFF-EEDGwz-FFkvVv'

#作業フォルダ名(gogleドライブ/Mydrive/Pokemon/の下に作業フォルダを作成)
dir_name = '白馬チラチーノ'

#ファイル名
list_name ="list.csv" #同じカードまとめた
raw_name  ="raw.csv"  #同じカードまとめてない
#firsthands_n.csv n回の初手7+1ドロー
#firstsides_n.csv n回のサイド
#firstdecks_n.csv n回のデッキに残っているカード

#シュミレーション回数
loop=100

Mounted at /content/drive


In [ ]:
##条件分岐対象##
#種ポケモン
basic_pokemons=['はくばバドレックスV','チラーミィ','デデンネGX','ミュウ']
#最初に並べたいポケモン
start_pokemons=['はくばバドレックスV']
start_pokemon_balls=['クイックボール','ポケモン通信']
#エネルギー
energies=['基本水エネルギー','たっぷりバケツ']
#アシストポケモン
assist_pokemons=['チラーミィ']
assist_pokemon_balls=['クイックボール','ポケモン通信','レベルボール']
#ドロサポ
supports=['博士の研究（マグノリア博士）','マリィ']
##デデクロ
dedekuros=['デデンネGX']
#まとめる
conditions=[basic_pokemons,start_pokemons,start_pokemon_balls,energies,assist_pokemons,assist_pokemon_balls,supports,dedekuros]

In [ ]:

#seleniumインストール＆ライブラリ
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

from google.colab import drive
from bs4 import BeautifulSoup
import requests
import pandas as pd
from IPython.display import Image,display_jpeg
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

     |████████████████████████████████| 911kB 7.9MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates In

In [ ]:
#作業ディレクトリ作成
tooldir = "/content/drive/MyDrive/pokemon/"
!mkdir -v $tooldir
workdir = "/content/drive/MyDrive/pokemon/" + dir_name
!mkdir -v $workdir
imagedir = workdir + "/image" 
!mkdir -v $imagedir

mkdir: cannot create directory ‘/content/drive/MyDrive/pokemon/’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/pokemon/白馬チラチーノ’: File exists
mkdir: cannot create directory ‘/content/drive/MyDrive/pokemon/白馬チラチーノ/image’: File exists


In [ ]:
#スクレイピング
d = webdriver.Chrome('chromedriver',options=options)
url ="https://www.pokemon-card.com/deck/confirm.html?viewMode=image&deckID="+deckcode
d.get(url)
soup = BeautifulSoup(d.page_source, 'html.parser')

imgBlocks = soup.select('.imgBlockArea')
cPoss = soup.select("[class='cPos nowrap']") 

for (imgBlock,cPos) in zip (imgBlocks,cPoss):
    #print(cPos.text)
    id   = imgBlock.img['id'].strip("img_")
    name = imgBlock.img['alt']
    num  = cPos.text.strip('枚')
    #print(id)
    #print(name)
    #print(num)

In [ ]:
# デッキリスト及び画像を保存する

# list_name(デッキリスト)の出力
df_list = pd.DataFrame([],columns=['id','name','num'])
filename = workdir + "/" +list_name
for (imgBlock,cPos) in zip (imgBlocks,cPoss):
    id   = imgBlock.img['id'].strip("img_")
    name = imgBlock.img['alt']
    num  = cPos.text.strip('枚')
    df_list = df_list.append({'id':id,'name':name,'num':num},ignore_index=True)
df_list.to_csv(filename, index=False)
print(filename + "を保存しました")

#デッキリストに入っていないカードを条件に設定していないか
for condition in conditions:
  for item in condition:
    if (item in df_list.values):
      pass
    else:
      print(item+'はデッキに入っていません')
      raise ValueError(item+'はデッキに入っていません')

# raw_name(同じカードをまとめてないリスト)の出力
df_list = pd.read_csv(filename)
df_raw = pd.DataFrame([],columns=['id','name'])
filename = workdir + "/" +raw_name
for row, item in df_list.iterrows():
  for i in range(item['num']):
    df_raw = df_raw.append({'id':item['id'],'name':item['name']},ignore_index=True)
df_raw.to_csv(filename, index=False)
print(filename + "を保存しました")

# カード画像を保存する
for imgBlock in imgBlocks:
  id   = imgBlock.img['id'].strip("img_")   
  img_url = 'https://www.pokemon-card.com/' + imgBlock.img['src']
  r = requests.get(img_url)
  filename = workdir+'/image/' + id +'.jpg'
  with open(filename,'wb') as file:
    file.write(r.content)
print("カード画像を保存しました")

/content/drive/MyDrive/pokemon/白馬チラチーノ/list.csvを保存しました
/content/drive/MyDrive/pokemon/白馬チラチーノ/raw.csvを保存しました
カード画像を保存しました


In [ ]:
#関数

#引き直しチェック
def basic_pokemon_check(df_hands):
  result = False
  for basic_pokemon in basic_pokemons:
    if (basic_pokemon in df_hands.values):
      result = True
  return result


In [ ]:
#n回シュミレーション
df_deck = df_raw
df_firsthands_id = pd.DataFrame([])
df_firsthands_name = pd.DataFrame([])
df_side_id = pd.DataFrame([])
df_side_name = pd.DataFrame([])
df_deck_id = pd.DataFrame([])
df_deck_name = pd.DataFrame([])
count_hikinaoshi = 0
for i in range(loop):
  #初期化
  df_deck = df_raw
  #初手の7枚を抽出し、ソートする
  while True:
    df_first = df_deck.sample(n=7).sort_index()
    if (basic_pokemon_check(df_first)):
      break
    else:
      count_hikinaoshi = count_hikinaoshi+1
  df_deck = df_deck.drop(df_first.index)
  #サイドを6枚抽し、ソートする
  df_side = df_deck.sample(n=6).sort_index()
  df_deck = df_deck.drop(df_side.index)
  #ドロー
  df_draw = df_deck.sample(n=1).sort_index()
  df_deck = df_deck.drop(df_draw.index)

  #初手とドローを結合
  df_first = pd.concat([df_first,df_draw]).reset_index(drop=True)
  df_side=df_side.reset_index(drop=True)
  df_deck=df_deck.reset_index(drop=True)

  #i回目の結果を保存
  df_firsthands_id[str(i+1)+"回目"]=df_first['id']
  df_firsthands_name[str(i+1)+"回目"]=df_first['name']
  df_side_id[str(i+1)+"回目"]=df_side['id']
  df_side_name[str(i+1)+"回目"]=df_side['name']
  df_deck_id[str(i+1)+"回目"]=df_deck['id']
  df_deck_name[str(i+1)+"回目"]=df_deck['name']

#初手リストを保存
filename= workdir + "/firsthands_" + str(loop) + ".csv"
df_firsthands_name.to_csv(filename, index=False)
#サイドリストを保存
filename= workdir + "/firstsides_" + str(loop) + ".csv"
df_side_name.to_csv(filename, index=False)
#デッキリストを保存
filename= workdir + "/firstdecks_" + str(loop) + ".csv"
df_deck_name.to_csv(filename, index=False)


In [ ]:
def start_pokemon_check(df_hands,df_deck,target,y_state,n_state):
  df_hands_tmp =df_hands[df_hands['state'] == target].drop('state',axis=1)
  for index,df_hand in df_hands_tmp.iterrows():
    check_result = False
    for num,value in df_hand.iteritems():
      #手札にエースポケモンがいるか
      if (value in start_pokemons):      
        check_result = True
      #手札にボールがあるかどうか
      if (value in start_pokemon_balls):
        if (start_pokemons[0] in df_deck.loc[index].values):
            check_result = True
      if (check_result):
        df_hands.loc[index,'state']=y_state
        df_hands.loc[index,num]='使用済み_'+value
        break
    #どちらの条件も満たさなかった場合事故
    if (check_result):
      pass
    else:
      df_hands.loc[index,'state']=n_state
  return df_hands

def enerugy_check(df_hands,df_deck,target,y_state,n_state):
  df_hands_tmp =df_hands[df_hands['state'] == target].drop('state',axis=1)
  for index,df_hand in df_hands_tmp.iterrows():
    check_result = False
    for num,value in df_hand.iteritems():
      #手札にエースポケモンがいるか
      if (value in energies):      
        check_result = True
      if (check_result):
        df_hands.loc[index,'state']=y_state
        df_hands.loc[index,num]='使用済み_'+value
        break
    #どちらの条件も満たさなかった場合事故
    if (check_result):
      pass
    else:
      df_hands.loc[index,'state']=n_state
  return df_hands

def assist_pokemon_check(df_hands,df_deck,target,y_state,n_state):
  df_hands_tmp =df_hands[df_hands['state'] == target].drop('state',axis=1)
  for index,df_hand in df_hands_tmp.iterrows():
    check_result = False
    count=0
    for num,value in df_hand.iteritems():
      #手札にアシストポケモンがいるか
      if (value in assist_pokemons):      
        check_result = True
        df_hands.loc[index,num]='使用済み_'+value
        count=count+1
      #手札にボールがあるかどうか
      if (value in assist_pokemon_balls):
        if (assist_pokemons[0] in df_deck.loc[index].values):
            check_result = True
            df_hands.loc[index,num]='使用済み_'+value
            count=count+1
    #どちらの条件も満たさなかった場合事故
    if (count>1):
      df_hands.loc[index,'state']=y_state
    else:
      df_hands.loc[index,'state']=n_state
  return df_hands

def dedekuro_check(df_hands,df_deck,target,y_state,n_state):
  df_hands_tmp =df_hands[df_hands['state'] == target].drop('state',axis=1)
  for index,df_hand in df_hands_tmp.iterrows():
    num_basic_pokemon=0
    num_dedekuro=0
    tmp=99
    tmp_name='name'
    for num,value in df_hand.iteritems():
      if (value in basic_pokemons or value=='使用済み_はくばバドレックスV'):
        num_basic_pokemon= num_basic_pokemon+1
      if (value in dedekuros):
        num_dedekuro= num_dedekuro+1
      #手札にボールがあるかどうか
      if (value in start_pokemon_balls):
        tmp=num
        tmp_name=value
    #最終チェック
    #デデンネスタートではない
    if (num_basic_pokemon!=num_dedekuro):
      if (num_dedekuro >=1):
        df_hands.loc[index,'state']=y_state
      else:
        if (tmp!=99):
          df_hands.loc[index,'state']=y_state
          df_hands.loc[index,tmp]='使用済み_'+tmp_name
    else:
      df_hands.loc[index,'state']=n_state
  return df_hands



In [ ]:
# フローチャート開始
df_firsthands_name.loc["state"]='0:初期値'
df_hands = df_firsthands_name.T.copy()
df_deck = df_deck_name.T.copy()
print("引き直し回数は"+str(count_hikinaoshi)+"回です")

#スタートポケモンチェック
df_hands = start_pokemon_check(df_hands,df_deck,'0:初期値','1:エースをおけた','100:事故')
#display(df_hands['state'].value_counts())
display(df_hands['state'].value_counts())
print()
#手張りチェック
df_hands = enerugy_check(df_hands,df_deck,'1:エースをおけた','2:手張りができた','100:事故')
#display(df_hands['state'].value_counts())
display(df_hands['state'].value_counts())
print()
#アシストポケモンチェック
df_hands = assist_pokemon_check(df_hands,df_deck,'2:手張りができた','102:最高','101:普通')
display(df_hands['state'].value_counts())
#display(df_hands['state'].value_counts())
print()

#デデクロチェック
df_hands = dedekuro_check(df_hands,df_deck,'100:事故','103:デデンネで回避','104:デデンネスタート')
display(df_hands['state'].value_counts())
print()

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
display(df_hands)

引き直し回数は41回です


1:エースをおけた    88
100:事故       12
Name: state, dtype: int64

2:手張りができた    70
100:事故       30
Name: state, dtype: int64

101:普通    35
102:最高    35
100:事故    30
Name: state, dtype: int64

101:普通          35
102:最高          35
103:デデンネで回避     17
100:事故          10
104:デデンネスタート     3
Name: state, dtype: int64

,0,1,2,3,4,5,6,7,state
1回目,はくばバドレックスVMAX,チラチーノ,チラーミィ,リセットスタンプ,博士の研究（マグノリア博士）,ボスの指令（サカキ）,基本水エネルギー,基本水エネルギー,100:事故
2回目,使用済み_はくばバドレックスV,使用済み_チラーミィ,使用済み_クイックボール,使用済み_ポケモン通信,使用済み_基本水エネルギー,基本水エネルギー,基本水エネルギー,リセットスタンプ,102:最高
3回目,使用済み_はくばバドレックスV,チラチーノ,ミュウ,ポケモン通信,ポケモン通信,使用済み_ポケモン通信,マリィ,レベルボール,103:デデンネで回避
4回目,使用済み_はくばバドレックスV,使用済み_レベルボール,ふうせん,博士の研究（マグノリア博士）,メロン,使用済み_基本水エネルギー,基本水エネルギー,マリィ,101:普通
5回目,使用済み_はくばバドレックスV,使用済み_クイックボール,使用済み_ポケモン通信,リセットスタンプ,博士の研究（マグノリア博士）,使用済み_基本水エネルギー,基本水エネルギー,博士の研究（マグノリア博士）,102:最高
6回目,使用済み_はくばバドレックスV,チラチーノ,デデンネGX,ボスの指令（サカキ）,ボスの指令（サカキ）,使用済み_基本水エネルギー,基本水エネルギー,リセットスタンプ,101:普通
7回目,使用済み_はくばバドレックスV,チラチーノ,使用済み_チラーミィ,ツールジャマー,マリィ,メロン,使用済み_基本水エネルギー,使用済み_チラーミィ,102:最高
8回目,使用済み_はくばバドレックスV,はくばバドレックスV,チラーミィ,デデンネGX,レベルボール,マリィ,ボスの指令（サカキ）,博士の研究（マグノリア博士）,103:デデンネで回避
9回目,使用済み_はくばバドレックスV,はくばバドレックスV,チラチーノ,使用済み_クイックボール,使用済み_たっぷりバケツ,たっぷりバケツ,基本水エネルギー,はくばバドレックスV,101:普通
10回目,使用済み_はくばバドレックスV,使用済み_クイックボール,使用済み_ポケモン通信,ツールジャマー,博士の研究（マグノリア博士）,使用済み_基本水エネルギー,基本水エネルギー,基本水エネルギー,102:最高


### **以下は、ソース修正中**

In [ ]:
def start_pokemon_check(df_hands,df_deck,check_target,check_target_state,check_num,y_state,n_state):
  df_hands_tmp =df_hands[df_hands[check_target] == check_target_state]
  for index,df_hand in df_hands_tmp.iterrows():
    check_result = False
    for num,value in df_hand.iteritems():
      #手札にエースポケモンがいるか
      if (value in start_pokemons):      
        check_result = True
      #手札にボールがあるかどうか
      if (value in start_pokemon_balls):
        if (start_pokemons[0] in df_deck.loc[index].values):
            check_result = True
      if (check_result):
        df_hands.loc[index,check_num]=y_state
        df_hands.loc[index,num]='使用済み_'+value
        break
    #どちらの条件も満たさなかった場合事故
    if (check_result):
      pass
    else:
      df_hands.loc[index,'state']=n_state
  return df_hands

def enerugy_check(df_hands,df_deck,target,y_state,n_state):
  df_hands_tmp =df_hands[df_hands['state'] == target].drop('state',axis=1)
  for index,df_hand in df_hands_tmp.iterrows():
    check_result = False
    for num,value in df_hand.iteritems():
      #手札にエースポケモンがいるか
      if (value in energies):      
        check_result = True
      if (check_result):
        df_hands.loc[index,'state']=y_state
        df_hands.loc[index,num]='使用済み_'+value
        break
    #どちらの条件も満たさなかった場合事故
    if (check_result):
      pass
    else:
      df_hands.loc[index,'state']=n_state
  return df_hands

def assist_pokemon_check(df_hands,df_deck,target,y_state,n_state):
  df_hands_tmp =df_hands[df_hands['state'] == target].drop('state',axis=1)
  for index,df_hand in df_hands_tmp.iterrows():
    check_result = False
    count=0
    for num,value in df_hand.iteritems():
      #手札にアシストポケモンがいるか
      if (value in assist_pokemons):      
        check_result = True
        df_hands.loc[index,num]='使用済み_'+value
        count=count+1
      #手札にボールがあるかどうか
      if (value in assist_pokemon_balls):
        if (assist_pokemons[0] in df_deck.loc[index].values):
            check_result = True
            df_hands.loc[index,num]='使用済み_'+value
            count=count+1
    #どちらの条件も満たさなかった場合事故
    if (count>1):
      df_hands.loc[index,'state']=y_state
    else:
      df_hands.loc[index,'state']=n_state
  return df_hands

def dedekuro_check(df_hands,df_deck,target,y_state,n_state):
  df_hands_tmp =df_hands[df_hands['state'] == target].drop('state',axis=1)
  for index,df_hand in df_hands_tmp.iterrows():
    num_basic_pokemon=0
    num_dedekuro=0
    tmp=99
    tmp_name='name'
    for num,value in df_hand.iteritems():
      if (value in basic_pokemons or value=='使用済み_はくばバドレックスV'):
        num_basic_pokemon= num_basic_pokemon+1
      if (value in dedekuros):
        num_dedekuro= num_dedekuro+1
      #手札にボールがあるかどうか
      if (value in start_pokemon_balls):
        tmp=num
        tmp_name=value
    #最終チェック
    #デデンネスタートではない
    if (num_basic_pokemon!=num_dedekuro):
      if (num_dedekuro >=1):
        df_hands.loc[index,'state']=y_state
      else:
        if (tmp!=99):
          df_hands.loc[index,'state']=y_state
          df_hands.loc[index,tmp]='使用済み_'+tmp_name
    else:
      df_hands.loc[index,'state']=n_state
  return df_hands


In [ ]:
# フローチャート開始
df_firsthands_name.loc['check1']='999:初期値'
df_hands = df_firsthands_name.T.copy()
df_deck = df_deck_name.T.copy()
print("引き直し回数は"+str(count_hikinaoshi)+"回です")

#スタートポケモンチェック
df_hands = start_pokemon_check(df_hands,df_deck,'check1','999:初期値','check1','1:エースをおけた','2:エースをおけない')
#display(df_hands['state'].value_counts())
display(pd.pivot_table(df_hands,index=['check1']
                       
                       ,aggfunc='count'))
print()

引き直し回数は41回です


TypeError: ignored

In [ ]:
# #ダミー画像作成
# def create_monochromatic_img(color, size):
#     r = color[0] * np.ones((size[1], size[0], 1), dtype=np.uint8)
#     g = color[1] * np.ones((size[1], size[0], 1), dtype=np.uint8)
#     b = color[2] * np.ones((size[1], size[0], 1), dtype=np.uint8)
#     return np.concatenate([r, g, b], axis=2)

# color = [255, 255, 255] #[r,g,b]
# size = [748,1044] #[height,width]
# img = create_monochromatic_img(color, size)
# #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)


In [ ]:
# #画像を表示
# n = int(loop/10)
# for i in range(n):
#   df_temp = df_firsthands_id.iloc[:,i*10:(i+1)*10]
#   infile=list(range(1,1000))
#   image=list(range(1,1000))

#   filename="test"
#   num = 1
#   for colname, item in df_temp.iteritems():
#     for index,value in item.iteritems():
#       if(index==7):
#         image[num] = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  
#         tmp = cv2.imread(imagedir + "/" + str(value) + ".jpg")
#         image[num+1] = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
#         num =num+2
#       else:
#         tmp = cv2.imread(imagedir + "/" + str(value) + ".jpg")
#         image[num] = cv2.cvtColor(tmp, cv2.COLOR_BGR2RGB)
#         num =num+1
#   imgname= workdir + "/firsthands_" +str(i)
#   plt.figure(figsize=(10, 20))
#   plt.subplots_adjust(wspace=0.0, hspace=0.0)
#   print(imgname+ '.jpg')
#   for i in range(1,91): 
#       #image[i] = cv2.imread(infile[i])
#       #image[i] = cv2.cvtColor(image[i], cv2.COLOR_BGR2RGB)
#       plt.subplot(10,9,i)
#       plt.subplots_adjust(hspace=0.01,wspace=-0)
#       plt.tick_params(labelbottom="off")
#       plt.tick_params(labelleft="off")
#       plt.tick_params(bottom="off")
#       plt.tick_params(left="off")
#       plt.gca().spines['right'].set_visible(False)
#       plt.gca().spines['top'].set_visible(False)
#       plt.gca().spines['left'].set_visible(False)
#       plt.gca().spines['bottom'].set_visible(False)
#       plt.rcParams["font.size"] = 20
#       plt.axis("off")
#       #plt.title( str(i))
#       plt.imshow(image[i]) 
#   plt.savefig(imgname + '.jpg')